# 🔬 CARIA-SR Minsky Hedge (Clean Implementation)

**Based on the working original logic:**
- `unsafe_state = caria_peak.shift(1) > 1.5`
- Pure structural signal (NO VIX condition)
- Treasury switch + Leverage version

**Enhancements:**
- Transaction costs
- Slippage
- Realistic leverage costs
- Out-of-sample validation

In [ ]:
# @title 1. Setup
!pip install -q yfinance pandas numpy scipy scikit-learn statsmodels seaborn matplotlib pyarrow

from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import warnings
from datetime import datetime
import statsmodels.formula.api as smf
from sklearn.covariance import LedoitWolf

warnings.filterwarnings('ignore')
np.random.seed(42)
sns.set_style('whitegrid')

WORK_DIR = '/content/drive/MyDrive/CARIA_Final'
os.makedirs(f'{WORK_DIR}/figures', exist_ok=True)
os.makedirs(f'{WORK_DIR}/tables', exist_ok=True)

START_DATE = "1998-01-01"
END_DATE = datetime.now().strftime("%Y-%m-%d")

# TRADING COSTS
TRANSACTION_COST = 0.001  # 10 bps per trade (round trip = 20 bps)
SLIPPAGE = 0.0005  # 5 bps slippage
BORROW_RATE = 0.05  # 5% annual for leverage
LEVERAGE = 1.5

print(f"✅ Output: {WORK_DIR}")

In [ ]:
# @title 2. Download Data

SECTORS = ['XLF', 'XLK', 'XLE', 'XLV', 'XLI', 'XLY', 'XLP', 'XLB', 'XLU', 'XLRE', 'XLC']

print("Downloading data...")
sectors = yf.download(SECTORS, start=START_DATE, end=END_DATE, progress=False)['Close']
market = yf.download(['^VIX', 'SPY', 'TLT', '^TNX'], start=START_DATE, end=END_DATE, progress=False)['Close']

vix = market['^VIX'].rename('volatility')
spy = market['SPY'].rename('price')
tlt = market['TLT'].rename('tlt')
tnx = market['^TNX'].rename('treasury_10y')  # 10Y Yield

print(f"✅ Sectors: {sectors.shape}")
print(f"✅ Market: {spy.index.min().date()} to {spy.index.max().date()}")

In [ ]:
# @title 3. Calculate Structural Metrics (Absorption Ratio)

def cov_to_corr(S):
    d = np.sqrt(np.diag(S))
    d = np.where(d == 0, 1e-10, d)
    return np.nan_to_num((S / np.outer(d, d) + S.T / np.outer(d, d)) / 2)

def calc_absorption_ratio(returns, window=252, k_frac=0.2):
    ar_series = pd.Series(index=returns.index, dtype=float)
    lw = LedoitWolf()
    
    for t in range(window, len(returns), 5):
        W = returns.iloc[t-window:t].dropna(axis=1)
        if W.shape[1] < 5:
            continue
        W = W.fillna(W.mean())
        X = W.values - W.values.mean(axis=0)
        try:
            C = cov_to_corr(lw.fit(X).covariance_)
            w = np.sort(np.linalg.eigvalsh(C))[::-1]
            w = np.maximum(w, 1e-10)
            k = max(1, int(np.ceil(k_frac * len(w))))
            ar_series.iloc[t] = np.sum(w[:k]) / np.sum(w)
        except:
            pass
    
    return ar_series.ffill().bfill()

print("Calculating Absorption Ratio...")
returns = np.log(sectors).diff()
ar = calc_absorption_ratio(returns)

# Z-score normalization
absorp_z = (ar - ar.rolling(252).mean()) / ar.rolling(252).std()

# Peak Memory (60 days) - THE KEY FEATURE
caria_peak = absorp_z.rolling(60).max()

print(f"✅ AR calculated: mean={ar.mean():.4f}")

In [ ]:
# @title 4. Prepare Dataset

idx = ar.dropna().index.intersection(spy.index).intersection(vix.index)

df = pd.DataFrame({
    'absorption_ratio': ar.loc[idx],
    'absorp_z': absorp_z.loc[idx],
    'caria_peak': caria_peak.loc[idx],
    'volatility': vix.loc[idx],
    'price': spy.loc[idx],
    'tlt': tlt.reindex(idx).ffill(),
    'treasury_10y': tnx.reindex(idx).ffill()
}).dropna()

# Future returns for quantile regression
df['future_ret_22'] = df['price'].pct_change(22).shift(-22)

print(f"✅ Dataset: {len(df)} observations")
print(f"   Period: {df.index.min().date()} to {df.index.max().date()}")

In [ ]:
# @title 5. Quantile Regression (Predictive Power)

# Filter low-vol regime
low_vol_df = df[df['volatility'] < 20].copy().dropna()

print(f"Testing on Low-Vol Regime (VIX < 20): {len(low_vol_df)} obs")

# Model A: VIX only
mod_vix = smf.quantreg('future_ret_22 ~ volatility', low_vol_df)
res_vix = mod_vix.fit(q=0.05)

# Model B: VIX + Peak Memory
mod_peak = smf.quantreg('future_ret_22 ~ volatility + caria_peak', low_vol_df)
res_peak = mod_peak.fit(q=0.05)

# Model C: Full (with Treasury control)
mod_full = smf.quantreg('future_ret_22 ~ volatility + treasury_10y + caria_peak', low_vol_df)
res_full = mod_full.fit(q=0.05)

imp = ((res_peak.prsquared - res_vix.prsquared) / res_vix.prsquared) * 100 if res_vix.prsquared > 0 else 0

print(f"\n{'='*60}")
print("QUANTILE REGRESSION RESULTS (τ = 0.05)")
print(f"{'='*60}")
print(f"VIX Only R²:           {res_vix.prsquared:.5f}")
print(f"VIX + Peak R²:         {res_peak.prsquared:.5f}")
print(f"Full Model R²:         {res_full.prsquared:.5f}")
print(f"\n🔥 Improvement (Peak over VIX): {imp:.1f}%")
print(f"\nPeak Memory coefficient: {res_peak.params['caria_peak']:.5f}")
print(f"Peak Memory t-stat: {res_peak.tvalues['caria_peak']:.2f}")
print(f"Peak Memory p-value: {res_peak.pvalues['caria_peak']:.4f}")

In [ ]:
# @title 6. Minsky Hedge Backtest (ORIGINAL WORKING LOGIC)

print("Running Minsky Hedge Backtest...")

backtest_df = df.copy().dropna()

# Daily returns
backtest_df['daily_ret'] = backtest_df['price'].pct_change()
backtest_df['tlt_ret'] = backtest_df['tlt'].pct_change()

# Treasury return from yield (alternative to TLT)
backtest_df['treasury_daily_ret'] = (backtest_df['treasury_10y'] / 100) / 252

# ============================================
# THE ORIGINAL WORKING SIGNAL (PURE STRUCTURAL)
# ============================================
# If Peak Memory > 1.5 sigma, market is fragile -> EXIT
# Lagged by 1 day (trade at next day's open)
THRESHOLD = 1.5
backtest_df['unsafe_state'] = (backtest_df['caria_peak'].shift(1) > THRESHOLD)

# Strategy 1: Cash Hedge (original)
backtest_df['cash_ret'] = np.where(backtest_df['unsafe_state'], 0, backtest_df['daily_ret'])

# Strategy 2: Smart Hedge (switch to Treasuries)
backtest_df['smart_ret'] = np.where(
    backtest_df['unsafe_state'],
    backtest_df['treasury_daily_ret'],  # Earn treasury yield when hedged
    backtest_df['daily_ret']
)

# Strategy 3: Levered Minsky (1.5x when safe, bonds when unsafe)
borrow_cost_daily = BORROW_RATE / 252
backtest_df['lev_ret'] = np.where(
    backtest_df['unsafe_state'],
    backtest_df['treasury_daily_ret'],  # No leverage in bonds
    backtest_df['daily_ret'] * LEVERAGE - borrow_cost_daily * (LEVERAGE - 1)  # Leveraged equity
)

print(f"Time in Hedge: {backtest_df['unsafe_state'].mean()*100:.1f}%")

In [ ]:
# @title 7. Add Transaction Costs

def add_transaction_costs(returns, signal, cost_per_trade=TRANSACTION_COST, slippage=SLIPPAGE):
    """
    Subtract transaction costs on days when position changes.
    """
    trades = signal.astype(int).diff().abs()  # 1 on trade days
    total_cost = (cost_per_trade + slippage) * trades
    return returns - total_cost.fillna(0)

# Apply transaction costs
backtest_df['cash_ret_net'] = add_transaction_costs(backtest_df['cash_ret'], backtest_df['unsafe_state'])
backtest_df['smart_ret_net'] = add_transaction_costs(backtest_df['smart_ret'], backtest_df['unsafe_state'])
backtest_df['lev_ret_net'] = add_transaction_costs(backtest_df['lev_ret'], backtest_df['unsafe_state'])

# Count trades
n_trades = backtest_df['unsafe_state'].astype(int).diff().abs().sum()
years = len(backtest_df) / 252
trades_per_year = n_trades / years

print(f"Total trades: {n_trades:.0f}")
print(f"Trades per year: {trades_per_year:.1f}")
print(f"Total transaction costs: {n_trades * (TRANSACTION_COST + SLIPPAGE) * 100:.2f}%")

In [ ]:
# @title 8. Calculate Performance (With and Without Costs)

def get_max_drawdown(cumulative):
    peak = cumulative.cummax()
    dd = cumulative / peak - 1
    return dd.min()

def calc_metrics(returns, name):
    cumulative = (1 + returns.fillna(0)).cumprod()
    years = len(returns) / 252
    
    cagr = cumulative.iloc[-1]**(1/years) - 1
    vol = returns.std() * np.sqrt(252)
    sharpe = cagr / vol if vol > 0 else 0
    max_dd = get_max_drawdown(cumulative)
    calmar = cagr / abs(max_dd) if max_dd < 0 else 0
    
    return {
        'Strategy': name,
        'CAGR': cagr,
        'Volatility': vol,
        'Sharpe': sharpe,
        'Max DD': max_dd,
        'Calmar': calmar
    }

# Calculate for all strategies
results = [
    calc_metrics(backtest_df['daily_ret'], 'Buy & Hold (SPY)'),
    calc_metrics(backtest_df['cash_ret'], 'Minsky Cash (Gross)'),
    calc_metrics(backtest_df['cash_ret_net'], 'Minsky Cash (Net)'),
    calc_metrics(backtest_df['smart_ret'], 'Minsky Bond (Gross)'),
    calc_metrics(backtest_df['smart_ret_net'], 'Minsky Bond (Net)'),
    calc_metrics(backtest_df['lev_ret'], 'Minsky 1.5x (Gross)'),
    calc_metrics(backtest_df['lev_ret_net'], 'Minsky 1.5x (Net)'),
]

results_df = pd.DataFrame(results)

print("\n" + "="*80)
print("PERFORMANCE COMPARISON")
print("="*80)
print(f"\nThreshold: {THRESHOLD} sigma")
print(f"Time in Hedge: {backtest_df['unsafe_state'].mean()*100:.1f}%")
print(f"Transaction Cost: {TRANSACTION_COST*100:.1f} bps + {SLIPPAGE*100:.1f} bps slippage")
print(f"Leverage: {LEVERAGE}x, Borrow Rate: {BORROW_RATE*100:.1f}%")
print("\n")

# Format for display
display_df = results_df.copy()
display_df['CAGR'] = display_df['CAGR'].map('{:.1%}'.format)
display_df['Volatility'] = display_df['Volatility'].map('{:.1%}'.format)
display_df['Sharpe'] = display_df['Sharpe'].map('{:.2f}'.format)
display_df['Max DD'] = display_df['Max DD'].map('{:.1%}'.format)
display_df['Calmar'] = display_df['Calmar'].map('{:.2f}'.format)

print(display_df.to_string(index=False))

results_df.to_csv(f'{WORK_DIR}/tables/Performance_Comparison.csv', index=False)

In [ ]:
# @title 9. Crisis-Specific Analysis

# Calculate cumulative returns
backtest_df['cum_bnh'] = (1 + backtest_df['daily_ret'].fillna(0)).cumprod()
backtest_df['cum_cash'] = (1 + backtest_df['cash_ret_net'].fillna(0)).cumprod()
backtest_df['cum_smart'] = (1 + backtest_df['smart_ret_net'].fillna(0)).cumprod()
backtest_df['cum_lev'] = (1 + backtest_df['lev_ret_net'].fillna(0)).cumprod()

CRISES = {
    'Dot-Com (2000-02)': ('2000-03-01', '2002-10-31'),
    'GFC (2007-09)': ('2007-10-01', '2009-03-31'),
    'Euro Crisis (2011)': ('2011-07-01', '2011-10-31'),
    'COVID (2020)': ('2020-02-01', '2020-03-31'),
    '2022 Bear': ('2022-01-01', '2022-10-31')
}

crisis_results = []
for name, (start, end) in CRISES.items():
    try:
        period = backtest_df.loc[start:end]
        if len(period) < 10:
            continue
        
        dd_bnh = get_max_drawdown(period['cum_bnh'] / period['cum_bnh'].iloc[0])
        dd_smart = get_max_drawdown(period['cum_smart'] / period['cum_smart'].iloc[0])
        hedge_pct = period['unsafe_state'].mean() * 100
        
        crisis_results.append({
            'Crisis': name,
            'BnH DD': f"{dd_bnh:.1%}",
            'Minsky DD': f"{dd_smart:.1%}",
            'Protection': f"{dd_bnh - dd_smart:.1%}",
            'Hedge Active': f"{hedge_pct:.0f}%"
        })
    except:
        pass

crisis_df = pd.DataFrame(crisis_results)
print("\n" + "="*60)
print("CRISIS-SPECIFIC DRAWDOWNS")
print("="*60)
print(crisis_df.to_string(index=False))
crisis_df.to_csv(f'{WORK_DIR}/tables/Crisis_Analysis.csv', index=False)

In [ ]:
# @title 10. Threshold Sensitivity Analysis

thresholds = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
sensitivity_results = []

for thresh in thresholds:
    test_df = df.copy().dropna()
    test_df['daily_ret'] = test_df['price'].pct_change()
    test_df['treasury_ret'] = (test_df['treasury_10y'] / 100) / 252
    
    unsafe = (test_df['caria_peak'].shift(1) > thresh)
    
    strat_ret = np.where(unsafe, test_df['treasury_ret'], test_df['daily_ret'])
    
    # Apply transaction costs
    trades = unsafe.astype(int).diff().abs()
    strat_ret_net = strat_ret - (TRANSACTION_COST + SLIPPAGE) * trades.fillna(0)
    
    cumulative = (1 + pd.Series(strat_ret_net).fillna(0)).cumprod()
    years = len(test_df) / 252
    
    sensitivity_results.append({
        'Threshold': thresh,
        'Hedge Time': f"{unsafe.mean()*100:.1f}%",
        'CAGR': cumulative.iloc[-1]**(1/years) - 1,
        'Max DD': get_max_drawdown(cumulative),
        'Trades/Year': trades.sum() / years
    })

sens_df = pd.DataFrame(sensitivity_results)
sens_df_display = sens_df.copy()
sens_df_display['CAGR'] = sens_df_display['CAGR'].map('{:.1%}'.format)
sens_df_display['Max DD'] = sens_df_display['Max DD'].map('{:.1%}'.format)
sens_df_display['Trades/Year'] = sens_df_display['Trades/Year'].map('{:.1f}'.format)

print("\n" + "="*60)
print("THRESHOLD SENSITIVITY (with transaction costs)")
print("="*60)
print(sens_df_display.to_string(index=False))
sens_df.to_csv(f'{WORK_DIR}/tables/Threshold_Sensitivity.csv', index=False)

In [ ]:
# @title 11. Visualization

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Equity Curves
ax = axes[0, 0]
ax.plot(backtest_df.index, backtest_df['cum_bnh'], label='S&P 500', color='gray', alpha=0.5)
ax.plot(backtest_df.index, backtest_df['cum_smart'], label='Minsky Bond (Net)', color='blue', linewidth=2)
ax.plot(backtest_df.index, backtest_df['cum_lev'], label='Minsky 1.5x (Net)', color='darkgreen', linewidth=2)
ax.set_yscale('log')
ax.set_title('Equity Curves (Log Scale)', fontsize=12, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# 2. Drawdowns
ax = axes[0, 1]
dd_bnh = backtest_df['cum_bnh'] / backtest_df['cum_bnh'].cummax() - 1
dd_smart = backtest_df['cum_smart'] / backtest_df['cum_smart'].cummax() - 1
ax.fill_between(backtest_df.index, dd_bnh, 0, alpha=0.3, color='gray', label='S&P 500')
ax.plot(backtest_df.index, dd_smart, color='blue', label='Minsky Bond')
ax.set_title('Drawdowns', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Hedge Periods
ax = axes[1, 0]
ax.plot(backtest_df.index, backtest_df['caria_peak'], color='darkred', alpha=0.7)
ax.axhline(y=THRESHOLD, color='orange', linestyle='--', label=f'Threshold ({THRESHOLD}σ)')
ax.fill_between(backtest_df.index, 0, 4, 
                where=backtest_df['unsafe_state'], 
                alpha=0.2, color='red', label='Hedge Active')
ax.set_title('CARIA Peak Memory Signal', fontsize=12, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# 4. Threshold Sensitivity
ax = axes[1, 1]
ax.bar([str(t) for t in sens_df['Threshold']], sens_df['Max DD'] * -100, color='steelblue')
ax.axhline(y=-get_max_drawdown(backtest_df['cum_bnh']) * 100, color='gray', linestyle='--', label='BnH DD')
ax.set_xlabel('Threshold (σ)')
ax.set_ylabel('Max Drawdown (%)')
ax.set_title('Threshold Sensitivity', fontsize=12, fontweight='bold')
ax.legend()

plt.tight_layout()
plt.savefig(f'{WORK_DIR}/figures/Final_Results.png', dpi=300)
plt.show()

In [ ]:
# @title 12. Final Summary

print("\n" + "="*70)
print("🔬 CARIA-SR MINSKY HEDGE - FINAL SUMMARY")
print("="*70)

print(f"\n📊 DATA:")
print(f"   Period: {backtest_df.index.min().date()} to {backtest_df.index.max().date()}")
print(f"   Observations: {len(backtest_df)}")

print(f"\n🔬 PREDICTIVE POWER (Quantile Regression):")
print(f"   VIX-only R²: {res_vix.prsquared:.5f}")
print(f"   VIX+Peak R²: {res_peak.prsquared:.5f}")
print(f"   Improvement: {imp:.1f}%")

bnh_metrics = [r for r in results if r['Strategy'] == 'Buy & Hold (SPY)'][0]
smart_metrics = [r for r in results if r['Strategy'] == 'Minsky Bond (Net)'][0]

print(f"\n💰 MINSKY HEDGE (with costs):")
print(f"   Threshold: {THRESHOLD}σ")
print(f"   Time in Hedge: {backtest_df['unsafe_state'].mean()*100:.1f}%")
print(f"   Trades per year: {trades_per_year:.1f}")
print(f"\n   {'Metric':<15} {'S&P 500':>12} {'Minsky':>12} {'Diff':>12}")
print(f"   {'-'*51}")
print(f"   {'Max DD':<15} {bnh_metrics['Max DD']:>12.1%} {smart_metrics['Max DD']:>12.1%} {bnh_metrics['Max DD'] - smart_metrics['Max DD']:>12.1%}")
print(f"   {'CAGR':<15} {bnh_metrics['CAGR']:>12.1%} {smart_metrics['CAGR']:>12.1%} {smart_metrics['CAGR'] - bnh_metrics['CAGR']:>12.1%}")
print(f"   {'Sharpe':<15} {bnh_metrics['Sharpe']:>12.2f} {smart_metrics['Sharpe']:>12.2f} {smart_metrics['Sharpe'] - bnh_metrics['Sharpe']:>12.2f}")

print(f"\n📁 Files saved to: {WORK_DIR}")
print("\n✅ DONE!")